In [1]:
import requests
from datetime import datetime
import datetime as dt
import time
import json
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import pandas_gbq
from prophet import Prophet
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from tqdm import tqdm
import time
import pygsheets
import datetime as dt
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
credentials = Credentials.from_service_account_file('C:/Users/leemn/OneDrive/Documents/Service Account/cb-prod-297913-d207de781afd.json')

In [3]:
df = pd.read_excel(r'C:/Users/leemn/Downloads/Send_Loop_Driver_Split.xlsx',
                   sheet_name='11_April')
df.head()

,onfleetPhone,name,surname,phone,onfleetPhone.1,mall,user_type,Loop_Email,Store and Name,Email,I am a,vehicle_type
0,27Y,Fabrice Tshisuaka,Lukusa,2.781266e+10,2.781266e+10,Nicolway,13.0,NaN,Nicolway Fabrice Tshisuaka Lukusa,NaN,Bike Driver,Bike
1,27685654298.0,Tumelo,Ralake,2.771331e+10,2.771331e+10,Kyalami Corner Shopping Centre,13.0,NaN,Kyalami Corner Shopping Centre Tumelo Ralake,NaN,Bike Driver,Bike
2,27630154687.0,Michael,Mazi,2.773969e+10,2.773969e+10,Kyalami Corner Shopping Centre,11.0,NaN,Kyalami Corner Shopping Centre Michael Mazi,NaN,Bike Driver,Bike
3,2727749214438.0,Zoe,Muananza,2.774921e+10,2.774921e+10,WW Dark Store,13.0,Zoe.muana10@gmail.com,"""V&A Waterfront Zoe Muananza \t""",Zoe.muana10@gmail.com,Bike Driver,Bike
4,27656335346.0,Samuel,Tshabalala,2.771343e+10,2.771343e+10,Eastgate Shopping Centre,13.0,Vumile06@gmail.com,Eastgate Shopping Centre Samuel Tshabalala,Vumile06@gmail.com,Bike Driver,Bike


In [4]:
df = df[df.Loop_Email.notnull()]

In [5]:
email_list = list(df.Loop_Email.unique())
#convert the list to a tuple
p = tuple(email_list)
query_drivers = '''
SELECT DISTINCT order_client_id AS client_id, 
                driver.id AS driver_id, 
                driver.name AS driver_name,
                driver.mobile_no AS driver_mobile,
                driver.employee_code AS employee_code,
                driver.email as driver_email

FROM `cb-prod-297913.cb_prod_analytics.cb_end_orders_in_trip` 
WHERE driver.email IN {} 
AND order_client_id IN ('B0l6eGBH0UB7wOhWzZ9J')
GROUP BY 1,2,3,4,5,6
'''.format(p)

df_email_list = pd.read_gbq(query_drivers, project_id='cb-prod-297913', 
                         dialect='standard', 
                         #progress_bar_type='tqdm_notebook',
                         credentials=credentials)
df_email_list.head()

,client_id,driver_id,driver_name,driver_mobile,employee_code,driver_email
0,B0l6eGBH0UB7wOhWzZ9J,L4BaTWWlkhbnlLRJ2ADH,Emmanuel Kangoma,+27652859578,WooliesDash221,doctaluwaba@gmail.com
1,B0l6eGBH0UB7wOhWzZ9J,5jZJNmjVs5TiAQwDpKhQ,Vicky Mulopo Tshibuabua,+27613409821,WooliesDash98,vickydav88@gmail.com
2,B0l6eGBH0UB7wOhWzZ9J,YYbztXFB48OH6ID3suey,Nonda Bernache,+27694000083,W-1023,nondakilay5@gmail.com
3,B0l6eGBH0UB7wOhWzZ9J,NKxweUeBdLbTLIUxiSBH,Prince Sola,+27733342090,WooliesDash463,nkomop99@gmail.com
4,B0l6eGBH0UB7wOhWzZ9J,BbIvJ21DgetylTmrq6J8,Moyo Kemson,+27767884064,W-1083,kemsonmoyo@gmail.com


In [6]:
df = df.rename(columns={'Loop_Email':'driver_email'})

In [7]:
df2 = df[['vehicle_type','driver_email']]
df_final_data = pd.merge(df2, df_email_list, how='inner', on=['driver_email'])
df_final_data.head()

,vehicle_type,driver_email,client_id,driver_id,driver_name,driver_mobile,employee_code
0,Bike,mashupejohn88@gmail.com,B0l6eGBH0UB7wOhWzZ9J,hkyVBvE1j6t7caoMJROr,John Mashupe,+27671852511,W-1098
1,Bike,mashupejohn88@gmail.com,B0l6eGBH0UB7wOhWzZ9J,hkyVBvE1j6t7caoMJROr,John Mashupe,+27671852511,W-1098
2,Car,ziyandapeter@gmail.com,B0l6eGBH0UB7wOhWzZ9J,dvkOs4nkilHFDf53bhEo,Ziyanda Peter,+27839984488,W-1029
3,Bike,zipporahb@gmail.com,B0l6eGBH0UB7wOhWzZ9J,EchdeE9RAybtClKvgQVU,Zipporah Bakali,+27694470884,WooliesDash167
4,Bike,zestpiousm@gmail.com,B0l6eGBH0UB7wOhWzZ9J,svOEjAnBUvddsdxfw1mf,Zest Makonde,+27789748665,WooliesDash149


In [10]:
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Push predictions to BigQuery >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
df_final_data.to_gbq(destination_table = 'cb-prod-297913.cb_prod_analytics.cb_driver_vehicle_type',
       project_id = 'cb-prod-297913',
       credentials = credentials,
       chunksize = 50000,
       #progress_bar = True,
       if_exists = 'replace')

100%|██████████| 1/1 [00:00<?, ?it/s]
